In [22]:
import numpy as np
from typing import Self
from functools import cache
from tqdm import tqdm
import json

In [23]:
class TicTacToeBoard:
    EMPTY_CELL, FIRST_PLAYER_CELL, SECOND_PLAYER_CELL = 0, 1, 2

    WIN_COMBOS = (
        (0, 1, 2),
        (3, 4, 5),
        (6, 7, 8),
        (0, 3, 6),
        (1, 4, 7),
        (2, 5, 8),
        (0, 4, 8),
        (2, 4, 6),
    )

    WIN_ARRAY = np.array(WIN_COMBOS, dtype=int)  # shape (8,3)

    def __init__(self):
        # NOTE: using EMPTY in case that value changes
        #       for whatever reason, but obv if EMPTY=0
        #       then this is equivalent to np.zeros((9,))
        self._state = np.ones((9,)) * TicTacToeBoard.EMPTY_CELL

    @property
    def state(self) -> tuple[int, ...]:
        return tuple(self._state.tolist())

    # NOTE: __eq__ and __hash__ need to be implemented for
    #       cache functionality to work 
    def __eq__(self, other: Self):
        return (self._state == other._state).all()
    
    def __hash__(self):
        return hash(self.state)

    @cache
    def player_to_move(self) -> int:
        return TicTacToeBoard.FIRST_PLAYER_CELL if len(self.available_cell_indices()) % 2 == 1 else TicTacToeBoard.SECOND_PLAYER_CELL

    def reset(self) -> None:
        self._state *= TicTacToeBoard.EMPTY_CELL

    @cache
    def available_cell_indices(self) -> tuple[int, ...]:
        return tuple((self._state == TicTacToeBoard.EMPTY_CELL).nonzero()[0].tolist())
    
    @cache
    def terminated(self) -> bool:
        return self.tied() or self.first_player_won() or self.second_player_won()

    @cache
    def tied(self) -> bool:
        return (self._state != TicTacToeBoard.EMPTY_CELL).all()

    @cache
    def first_player_won(self) -> bool:
        return self.player_won(player=TicTacToeBoard.FIRST_PLAYER_CELL)

    @cache
    def second_player_won(self) -> bool:
        return self.player_won(player=TicTacToeBoard.SECOND_PLAYER_CELL)

    @cache
    def player_won(self, player: int) -> bool:
        cells = self._state[TicTacToeBoard.WIN_ARRAY]
        return bool(np.any(np.all(cells == player, axis=1)))

    @cache
    def transition(self, idx: int) -> Self:
        if self.terminated():
            raise RuntimeError("move attempted on completed game")

        if idx not in self.available_cell_indices():
            raise RuntimeError("illegal move")
        new_board = self.__class__()
        new_board._state = self._state.copy()
        new_board._state[idx] = self.player_to_move()
        return new_board
    
    def display(self):
        keys = {TicTacToeBoard.FIRST_PLAYER_CELL: "X", TicTacToeBoard.SECOND_PLAYER_CELL: "O", TicTacToeBoard.EMPTY_CELL: "_"}
        for i in range(0, 9, 3):
            print(f"{keys[self._state[i]]} {keys[self._state[i+1]]} {keys[self._state[i+2]]}")
        print()

In [24]:
AGENT_CELL, OPPONENT_CELL, EMPTY_CELL = "A", "O", "_"
# NOTE: ties are considered same as default value
WON_VALUE, LOST_VALUE, DEFAULT_VALUE = 1.0, 0.0, 0.5

@cache
def serialize_board(board: TicTacToeBoard, player: int) -> str:
    return "".join([
        EMPTY_CELL if c == TicTacToeBoard.EMPTY_CELL
        else (AGENT_CELL if c == player else OPPONENT_CELL)
        for c in board.state
    ])


def populate_value_if_non_existent(value_table: dict, board: TicTacToeBoard, player: int):
    key = serialize_board(board, player=player)
    if key not in value_table:
        value_table[key] = DEFAULT_VALUE
    if board.first_player_won():
        value_table[key] = (WON_VALUE if player == TicTacToeBoard.FIRST_PLAYER_CELL else LOST_VALUE)
    elif board.second_player_won():
        value_table[key] = (WON_VALUE if player == TicTacToeBoard.SECOND_PLAYER_CELL else LOST_VALUE)

def get_value(value_table: dict, board: TicTacToeBoard, player: int):
    populate_value_if_non_existent(value_table, board, player)
    return value_table[serialize_board(board, player)]

def sample_policy(
    value_table: dict,
    board: TicTacToeBoard,
    player: int,
    eps: float = 0.0
) -> tuple[int, bool]:
    """
    Greedy (with eps) over the VALUE of the next *decision* state,
    i.e. after you move AND the opponent has replied—without recursion.
    """
    # identify opponent
    other = (TicTacToeBoard.SECOND_PLAYER_CELL
             if player == TicTacToeBoard.FIRST_PLAYER_CELL
             else TicTacToeBoard.FIRST_PLAYER_CELL)

    # make sure current board is initialized
    populate_value_if_non_existent(value_table, board, player)

    candidates: list[tuple[int, float]] = []
    for idx in board.available_cell_indices():
        # 1) simulate your move
        b1 = board.transition(idx)

        # 2) simulate opponent's greedy reply (no eps) at b1
        if not b1.terminated():
            # ensure b1 is in the table for opponent's POV
            populate_value_if_non_existent(value_table, b1, other)

            # find opponent's best reply
            best_val = None
            best_opp_idx = None
            for opp_idx in b1.available_cell_indices():
                b2_opp = b1.transition(opp_idx)
                populate_value_if_non_existent(value_table, b2_opp, other)
                v_opp = get_value(value_table, b2_opp, other)
                if best_val is None or v_opp > best_val:
                    best_val = v_opp
                    best_opp_idx = opp_idx
            # apply that reply
            b2 = b1.transition(best_opp_idx)
        else:
            # game ended on your move
            b2 = b1

        # 3) now b2 is the board at your next turn (or terminal)
        populate_value_if_non_existent(value_table, b2, player)
        v_next = get_value(value_table, b2, player)
        candidates.append((idx, v_next))

    # 4) pick greedy action
    greedy_idx, _ = max(candidates, key=lambda x: x[1])

    # 5) epsilon‐exploration
    if len(candidates) > 1 and np.random.rand() < eps:
        other_idxs = [i for i, _ in candidates if i != greedy_idx]
        return np.random.choice(other_idxs), False

    return greedy_idx, True



def update_value(value_table: dict, last_board: TicTacToeBoard, current_board: TicTacToeBoard, player: int, learning_rate: float):
    key_last = serialize_board(last_board, player)
    # these two calls will auto‐create missing entries 
    v_last = get_value(value_table, last_board, player)
    v_curr = get_value(value_table, current_board, player)
    value_table[key_last] = v_last + learning_rate * (v_curr - v_last)


In [25]:
def save(value_table, path):
    # dump keys as strings for JSON
    export = {"".join(str(x) for x in k): v for k, v in value_table.items()}
    with open(path, "w") as f:
        json.dump(export, f)

def load(path):
    raw = json.load(open(path, "r"))
    return {tuple(int(c) for c in k): v for k, v in raw.items()}

In [26]:
# NOTE: self play loop, agent plays both first and second player, changing its
#       role after each turn

FIRST_PLAYER_WON="first player won"
SECOND_PLAYER_WON="second player won"
TIE="tie"

def random_opponent(board: TicTacToeBoard, player: int) -> tuple[int, bool]:
    """Pick a random legal move. Always non‑greedy."""
    idx = np.random.choice(board.available_cell_indices())
    return idx, False

def train_selfplay(
    value_table: dict,
    n_episodes: int,
    alpha0: float,
    min_alpha: float,
    alpha_decay: float,
    eps0: float,
    min_eps: float,
    eps_decay: float,
):
    """Self‐play: both sides use sample_policy (ε‐greedy), updating on every greedy move."""
    alpha, epsilon = alpha0, eps0

    for _ in tqdm(range(n_episodes), desc="Self‑play"):
        board = TicTacToeBoard()
        p1_last = p2_last = None
        p1_greedy = p2_greedy = False

        while not board.terminated():
            # -- Player 1 --
            idx, p1_greedy = sample_policy(value_table, board,
                                           player=TicTacToeBoard.FIRST_PLAYER_CELL,
                                           eps=epsilon)
            p1_last, board = board, board.transition(idx)
            if p2_greedy:
                update_value(value_table, p2_last, board,
                             TicTacToeBoard.SECOND_PLAYER_CELL, alpha)
                p2_greedy = False
            if board.terminated(): break

            # -- Player 2 --
            idx, p2_greedy = sample_policy(value_table, board,
                                           player=TicTacToeBoard.SECOND_PLAYER_CELL,
                                           eps=epsilon)
            p2_last, board = board, board.transition(idx)
            if p1_greedy:
                update_value(value_table, p1_last, board,
                             TicTacToeBoard.FIRST_PLAYER_CELL, alpha)
                p1_greedy = False

        # Final TD update for whoever moved last
        if p1_greedy:
            update_value(value_table, p1_last, board,
                         TicTacToeBoard.FIRST_PLAYER_CELL, alpha)
        if p2_greedy:
            update_value(value_table, p2_last, board,
                         TicTacToeBoard.SECOND_PLAYER_CELL, alpha)

        # decay α and ε
        alpha   = max(min_alpha, alpha * alpha_decay)
        epsilon = max(min_eps,   epsilon * eps_decay)


def train_against_random(
    value_table: dict,
    n_episodes: int,
    alpha0: float,
    min_alpha: float,
    alpha_decay: float,
    eps0: float,
    min_eps: float,
    eps_decay: float,
    agent_plays_first: bool
):
    """
    Agent (with TD learning) vs random opponent.
    If agent_plays_first=True, agent is Player 1; else agent is Player 2.
    """
    alpha, epsilon = alpha0, eps0
    agent = (TicTacToeBoard.FIRST_PLAYER_CELL
             if agent_plays_first else TicTacToeBoard.SECOND_PLAYER_CELL)

    for _ in tqdm(range(n_episodes),
                  desc=f"Agent vs Random ({'1st' if agent_plays_first else '2nd'})"):
        board = TicTacToeBoard()

        last_board = None
        last_greedy = False
        current = TicTacToeBoard.FIRST_PLAYER_CELL

        while not board.terminated():
            if current == agent:
                # agent’s turn: ε‑greedy on V
                idx, last_greedy = sample_policy(value_table, board,
                                                 player=current, eps=epsilon)
                prev = board
                board = board.transition(idx)
                if last_greedy:
                    update_value(value_table, prev, board, current, alpha)
            else:
                # random opponent
                idx, _ = random_opponent(board, current)
                board = board.transition(idx)
                last_greedy = False

            current = (TicTacToeBoard.SECOND_PLAYER_CELL
                       if current == TicTacToeBoard.FIRST_PLAYER_CELL
                       else TicTacToeBoard.FIRST_PLAYER_CELL)

        # final update if agent moved last
        if last_greedy and current != agent:
            # `last_board` holds the board before agent’s last move
            update_value(value_table, prev, board, agent, alpha)

        # decay α and ε
        alpha   = max(min_alpha, alpha * alpha_decay)
        epsilon = max(min_eps,   epsilon * eps_decay)

In [27]:
np.random.seed(42)

In [28]:
# parameters
alpha0     = 0.1
min_alpha  = 0.01
alpha_decay= 0.995
eps0       = 0.1
min_eps    = 0.01
eps_decay  = 0.995

value_table = {}

# 1) self‑play
train_selfplay(
    value_table=value_table,
    n_episodes=100_000,
    alpha0=alpha0,      min_alpha=min_alpha,
    alpha_decay=alpha_decay,
    eps0=eps0,          min_eps=min_eps,
    eps_decay=eps_decay
)

# 2) agent as first vs random
train_against_random(
    value_table=value_table,
    n_episodes=50_000,
    alpha0=alpha0,      min_alpha=min_alpha,
    alpha_decay=alpha_decay,
    eps0=eps0,          min_eps=min_eps,
    eps_decay=eps_decay,
    agent_plays_first=True
)

# 3) agent as second vs random
train_against_random(
    value_table=value_table,
    n_episodes=50_000,
    alpha0=alpha0,      min_alpha=min_alpha,
    alpha_decay=alpha_decay,
    eps0=eps0,          min_eps=min_eps,
    eps_decay=eps_decay,
    agent_plays_first=False
)

Agent vs Random (2nd): 100%|██████████| 50000/50000 [01:01<00:00, 818.80it/s]


In [29]:
def play_game(value_table: dict, eps: float = 0.0) -> str:
    """Plays one game greedily with fixed V-table and returns outcome."""
    board  = TicTacToeBoard()
    player = TicTacToeBoard.FIRST_PLAYER_CELL

    while not board.terminated():
        # always greedy here
        action, _ = sample_policy(
            value_table, board,
            player=player, eps=eps
        )
        board = board.transition(action)
        # switch sides
        player = (TicTacToeBoard.SECOND_PLAYER_CELL
                  if player == TicTacToeBoard.FIRST_PLAYER_CELL
                  else TicTacToeBoard.FIRST_PLAYER_CELL)

    # return one of FIRST_PLAYER_WON, SECOND_PLAYER_WON, or TIE
    if board.first_player_won():
        return FIRST_PLAYER_WON
    elif board.second_player_won():
        return SECOND_PLAYER_WON
    else:
        return TIE


# Run a no‑exploration evaluation
results = {FIRST_PLAYER_WON: 0,
           SECOND_PLAYER_WON: 0,
           TIE: 0}

for _ in tqdm(range(10_000)):
    outcome = play_game(value_table, eps=0.0)
    results[outcome] += 1

print("Greedy evaluation (no eps):", results)


100%|██████████| 10000/10000 [00:15<00:00, 662.87it/s]

Greedy evaluation (no eps): {'first player won': 0, 'second player won': 0, 'tie': 10000}


In [30]:
def sample_eval_policy(board: TicTacToeBoard) -> int:
    player_to_move = board.player_to_move()
    avail = board.available_cell_indices()
    if len(avail) == 0:
        raise ValueError("No moves left")
    for c in avail:
        if board.transition(c).player_won(player_to_move):
            return c
    return int(np.random.choice(avail))

In [31]:
def eval_policy(value_table, n_episodes, player):
    outcomes = {FIRST_PLAYER_WON: 0, SECOND_PLAYER_WON: 0, TIE: 0}
    # TODO: same as above training loop, make loop cleaner using cycle to switch through current player or somethign
    for i in tqdm(range(n_episodes)):
        current = TicTacToeBoard()

        while True:
            action = None
            if player == TicTacToeBoard.FIRST_PLAYER_CELL:
                action, _ = sample_policy(value_table=value_table, board=current, player=player, eps=0.0)
            else:
                action = sample_eval_policy(board=current)
            current = current.transition(action)

            # TODO: change to use logging
            # current.display()

            if current.terminated():
                break


            if player == TicTacToeBoard.SECOND_PLAYER_CELL:
                action, _ = sample_policy(value_table=value_table, board=current, player=player, eps=0.0)
            else:
                action = sample_eval_policy(board=current)
            current.transition(action)
            # TODO: change to use logging
            # current.display()
            if current.terminated():
                break
        
        assert current.terminated(), "should be terminated at end of episode"
        outcomes[FIRST_PLAYER_WON if current.first_player_won() else SECOND_PLAYER_WON if current.second_player_won() else TIE] += 1

    return outcomes

        

In [32]:
eval_policy(value_table=value_table, n_episodes=10000, player=TicTacToeBoard.FIRST_PLAYER_CELL)

100%|██████████| 10000/10000 [00:13<00:00, 721.12it/s]


{'first player won': 10000, 'second player won': 0, 'tie': 0}

In [33]:
eval_policy(value_table=value_table, n_episodes=10000, player=TicTacToeBoard.SECOND_PLAYER_CELL)

100%|██████████| 10000/10000 [00:21<00:00, 468.53it/s]


{'first player won': 6880, 'second player won': 2678, 'tie': 442}

In [34]:
save(value_table=value_table, path="table.json")

In [35]:
def serialize_to_ui_format(value_table):
    return {"".join(["1" if c == AGENT_CELL else "2" if c == OPPONENT_CELL else "0" for c in s]): v for s, v in value_table.items()}

save(value_table=serialize_to_ui_format(value_table=value_table), path="ui_value_table.json")